In [14]:
import os
import shutil

MAINPATH = "D:\\PSID_ACTIVELEARNING_FRAMEWORK\\00_quick_draft"
FOLDERLIST = ["Ar", "CO2", "H2", "He", "N2", "O2"]

# 결과 저장 경로 (MAINPATH 밑에 Results라는 폴더 생성)
OUTPUT_DIR = os.path.join(MAINPATH, "Collected_Files")
os.makedirs(OUTPUT_DIR, exist_ok=True)

for folder in FOLDERLIST:
    base_path = os.path.join(MAINPATH, folder)
    # 대상 폴더만 필터링
    target_folder = [
        os.path.join(base_path, x)
        for x in os.listdir(base_path)
        if (
            "chunk" not in x
            and os.path.isdir(os.path.join(base_path, x))
            and x != "time_below_50"
            and x != "Using_force_field_def_20250811_14H_BH"
        )
    ]

    for target in target_folder:
        chunks_path = [
            os.path.join(target, x)
            for x in os.listdir(target)
            if "chunk" in x
        ]

        for chunk in chunks_path:
            # MAINPATH 이후 경로 추출 → \로 split → _로 join
            relative_path = os.path.relpath(chunk, MAINPATH)
            folder_name = "_".join(relative_path.split(os.sep))

            # 저장할 폴더 경로
            save_dir = os.path.join(OUTPUT_DIR, folder_name)
            os.makedirs(save_dir, exist_ok=True)

            # 복사할 파일 목록
            for fname in ["07result.csv", "99_progress.log"]:
                src_file = os.path.join(chunk, fname)
                if os.path.exists(src_file):
                    dst_file = os.path.join(save_dir, fname)
                    shutil.copy2(src_file, dst_file)
                    print(f"Copied: {src_file} -> {dst_file}")
                else:
                    print(f"File not found: {src_file}")


Copied: D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Ar\time_below_50_273_0.01\chunk_1\07result.csv -> D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Collected_Files\Ar_time_below_50_273_0.01_chunk_1\07result.csv
Copied: D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Ar\time_below_50_273_0.01\chunk_1\99_progress.log -> D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Collected_Files\Ar_time_below_50_273_0.01_chunk_1\99_progress.log
Copied: D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Ar\time_below_50_273_0.05\chunk_1\07result.csv -> D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Collected_Files\Ar_time_below_50_273_0.05_chunk_1\07result.csv
Copied: D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Ar\time_below_50_273_0.05\chunk_1\99_progress.log -> D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Collected_Files\Ar_time_below_50_273_0.05_chunk_1\99_progress.log
Copied: D:\PSID_ACTIVELEARNING_FRAMEWORK\00_quick_draft\Ar\time_below_50_273_0.1\chunk_1\07result.csv -> D:\PSID_ACTIVEL

In [37]:
import os
import pandas as pd

MAINPATH = "./Collected_Files/"
folder_list = os.listdir(MAINPATH)

OUTPUT_DIR = "Grouped_Results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

gas_data = {}

for folder in folder_list:
    folder_path = os.path.join(MAINPATH, folder)
    csv_file = os.path.join(folder_path, "07result.csv")
    if not os.path.exists(csv_file):
        continue

    df = pd.read_csv(csv_file)
    if "abs_mol_per_kg_framework" not in df.columns:
        continue

    # ── 폴더 이름 파싱 (gas_temp_pressure_chunk) ──────────────────────
    parts = folder.split("_")
    gas, temp, pressure = parts[0], parts[1], parts[2]

    # label (chunk 무시)
    label = f"{gas}_{temp}_{pressure}"

    # ── 필요한 데이터 선택 ──────────────────────
    df_small = df[["name", "abs_mol_per_kg_framework"]].copy()
    df_small = df_small.rename(columns={"abs_mol_per_kg_framework": label})

    # 기체별 데이터 누적 (세로 concat)
    if gas not in gas_data:
        gas_data[gas] = [df_small]
    else:
        gas_data[gas].append(df_small)

# ── 기체별 wide-format 저장 ──────────────────────
for gas, df_list in gas_data.items():
    merged = pd.concat(df_list, ignore_index=True)
    wide = merged.pivot_table(index="name", aggfunc="first")

    # ---- 열 순서 재정렬 (temp → pressure 순서) ----
    def parse_key(col):
        # 예: "Ar_273_15" → (273, 15.0)
        parts = col.split("_")
        temp = int(parts[1])
        pressure = float(parts[2])
        return (temp, pressure)

    sorted_cols = sorted(wide.columns, key=parse_key)
    wide = wide[sorted_cols]

    # 저장
    output_file = os.path.join(OUTPUT_DIR, f"{gas}_results.csv")
    wide.to_csv(output_file)
    print(f"Saved {output_file} with shape {wide.shape}")


Saved Grouped_Results\Ar_results.csv with shape (8267, 14)
Saved Grouped_Results\H2_results.csv with shape (8827, 2)
Saved Grouped_Results\He_results.csv with shape (10788, 14)
Saved Grouped_Results\N2_results.csv with shape (7026, 2)
Saved Grouped_Results\O2_results.csv with shape (7113, 2)


In [29]:
os.listdir("./Collected_Files/")

['Ar_273_0.01_chunk_1',
 'Ar_273_0.05_chunk_1',
 'Ar_273_0.1_chunk_1',
 'Ar_273_0.5_chunk_1',
 'Ar_273_15_chunk_1',
 'Ar_273_1_chunk_1',
 'Ar_273_5_chunk_1',
 'Ar_313_0.01_chunk_1',
 'Ar_313_0.05_chunk_1',
 'Ar_313_0.1_chunk_1',
 'Ar_313_0.5_chunk_1',
 'Ar_313_15_chunk_1',
 'Ar_313_15_chunk_2',
 'Ar_313_15_chunk_3',
 'Ar_313_1_chunk_1',
 'Ar_313_5_chunk_1',
 'H2_293_0.01_chunk_1',
 'H2_293_0.01_chunk_2',
 'H2_293_0.01_chunk_3',
 'H2_293_0.01_chunk_4',
 'H2_293_5_chunk_1',
 'H2_293_5_chunk_2',
 'H2_293_5_chunk_3',
 'H2_293_5_chunk_4',
 'He_273_15_chunk_1',
 'He_273_15_chunk_2',
 'He_273_15_chunk_3',
 'He_273_15_chunk_4',
 'He_273_1_chunk_1',
 'He_273_1_chunk_2',
 'He_273_1_chunk_3',
 'He_273_1_chunk_4',
 'He_273_5_chunk_1',
 'He_273_5_chunk_2',
 'He_273_5_chunk_3',
 'He_273_5_chunk_4',
 'He_313_15_chunk_1',
 'He_313_15_chunk_2',
 'He_313_15_chunk_3',
 'He_313_15_chunk_4',
 'He_313_15_chunk_5',
 'He_313_1_chunk_1',
 'He_313_1_chunk_2',
 'He_313_1_chunk_3',
 'He_313_1_chunk_4',
 'He_313_1